### Hints:
* Use Splinter to navigate the sites when needed and BeautifulSoup to help find and parse out the necessary data.
* Use Pymongo for CRUD applications for your database. For this homework, you can simply overwrite the existing document each time the /scrape url is visited and new data is obtained.

## Gretelnotes
think about simplifying all the browser to soup commands to a single line

## Setup

In [168]:
# Import Dependencies
import pandas as pd
import requests
import pymongo
from splinter import Browser
from bs4 import BeautifulSoup as bs

## NASA Mars News

In [169]:
# NOT NEEDED HERE
# Initialize PyMongo to work with MongoDBs
# conn = 'mongodb://localhost:27017'
# client = pymongo.MongoClient(conn)

# # Define database and collection
# db = client.mars_db
# collection = db.items

In [170]:
# Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. 

executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

url_news = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url_news)

html_news = browser.html
soup_news = bs(html_news, "html.parser")
    
# Latest News Title
#IS THERE A BETTER WAY OF DOING THIS? Maybe another place to find title or a way to capitalize first letter
news_title_html = soup_news.find("div", class_='image_and_description_container').find("a")["href"]
news_title = " ".join(news_title_html.split("/")[3].split("-"))

# Text for latest news 
news_p = soup_news.find("div", class_="rollover_description_inner").text


In [171]:
print(news_title)
print(news_p)

nasa social media and websites win webby awards
NASA's social media presence, the InSight mission social media accounts, NASA.gov and SolarSystem.NASA.gov will be honored at the 2019 Webby Awards - "the Oscars of the Internet."


## JPL Mars Space Images - Featured Image

In [172]:
# Find the image url for the current Featured Mars Image

executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

url_JPL = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url_JPL)

html_JPL = browser.html
soup_JPL = bs(html_JPL, "html.parser")

image_JPL = soup_JPL.find("article").find("footer").find("a")["data-fancybox-href"]
featured_image_url = f"https://www.jpl.nasa.gov{image_JPL}"

In [173]:
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA19346_ip.jpg


## Mars Weather

In [174]:
# Find the latest Mars weather tweet and save weatehr report text

executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

url_tweet = "https://twitter.com/marswxreport?lang=en"
browser.visit(url_tweet)

html_tweet = browser.html
soup_tweet = bs(html_tweet, "html.parser")

# Pull out text from tweet and format
tweet_text = soup_tweet.find("div", class_="js-tweet-text-container").find("p").text
mars_weather = " ".join(tweet_text.split("pic")[0].split("InSight ")[1].split("\n"))

In [175]:
print(mars_weather)

sol 145 (2019-04-24) low -98.1ºC (-144.6ºF) high -19.3ºC (-2.8ºF) winds from the SW at 4.4 m/s (9.8 mph) gusting to 11.6 m/s (26.1 mph) pressure at 7.40 hPa


## Mars Facts
* Visit the Mars Facts webpage here https://space-facts.com/mars/ and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
* Use Pandas to convert the data to a HTML table string.

In [176]:
# Unsure about the format I need to put this in. Parameter as index? Or as column headings?
# HTML definitely needs cleaning up

# Convert table of facts from url into pandas dataframe
url_facts = "https://space-facts.com/mars/"
facts_df = pd.read_html(url_facts)[0]

# Reformat dataframe
facts_df = facts_df.rename(columns={0: "Parameter" , 1: "Values"})
facts_df = facts_df.set_index("Parameter")

# Convert dataframe to htlp and clean up newlines
html_facts = facts_df.to_html().replace('\n', "")

In [177]:
print(html_facts)

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Values</th>    </tr>    <tr>      <th>Parameter</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.52 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-153 to 20 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>


## Mars Hemispheres

Get the name and url of a high resolution image for each hemisphere of Mars.

In [178]:
# Use splinter to access website through chrome

executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

url_hemi = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url_hemi)

# Create Beautiful soup object
soup_hemi = bs(browser.html, "html.parser")

In [179]:
# Define function

def get_hemi_image(x):
    """This function finds the name and url string for the full resolution 
    image of a hemisphere based on its index in a list and returns a dictionary of these values"""
    
    # Get name of hemisphere
    name_hemi = hemi_all[x].find("h3").text

    # Get link to page with full resolution image
    image_hemi = hemi_all[x].find("a")["href"]
    image_hemi_url = f"https://astrogeology.usgs.gov/{image_hemi}"
    
    # Open browser to selected hemisphere's page
    browser.visit(image_hemi_url)
    soup_hemi_img = bs(browser.html)

    # Get link to full resolution image
    hemi_full_url = soup_hemi_img.find("div", class_="downloads").find_all("li")[1].find("a")["href"]

    # Open browser to previous page
    browser.click_link_by_partial_text('Back')
    
    #return dictionary of values
    hemi_dict = {"title": name_hemi, "img_url": hemi_full_url}
    return(hemi_dict)

In [180]:
# Create list of hemisphere name and urls for each hemisphere in dictionary form

hemisphere_image_urls = []

hemi_all = soup_hemi.find_all("div", class_="description")
for x in range(4):
    hemisphere_image_urls.append(get_hemi_image(x))

In [181]:
print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif'}]
